In [20]:
"""module for defining input parameters"""

import numpy as np

# input node matrix, where each row represents the [x,y] coordinates of that node
NODE = np.array([
    [0,0], 
    [0,2], 
    [0,4],
    [3,0],
    [3,2],
    [3,4],
    [6,2],
    [6,4],
    [9,2],
    [9,4],
    [12,0],
    [12,2],
    [12,4],
    [15,0],
    [15,2],
    [15,4] 
    ])

tnnd = len(NODE) # total number of nodes

# input element matrix that establishes element-node connectivity
# each row represents an element: [bottom left, bottom right, top right, top left] 
ELEM = np.array([
    [1,4,5,2], 
    [2,5,6,3], 
    [5,7,8,6],
    [7,9,10,8],
    [9,12,13,10],
    [12,15,16,13],
    [11,14,15,12]  
])

tnel = len(ELEM) # total number of elements

# Young's Modulus of each element
E = np.array([
    10e6,
    10e6,
    10e6,
    10e6,
    10e6,
    10e6,
    10e6
])

# Poisson's ratio of each element
Nu = np.array([
    .3,
    .3,
    .3,
    .3,
    .3,
    .3,
    .3
])

# thickness of 2D element
t = .1

In [21]:
"""module for computing local stiffness matrices"""

import input # import input module with NODE, ELEM matrices and other parameters
import sympy as sp
import numpy as np

def local_stiffness(elem_num):
    """
    calculate the local stiffness matrix for a given element number.

    args:
        elem_num (int): the element number of interest
    
    returns:
        numpy.ndarray: the local stiffness matrix for the element of interest
    """
    # material properites and element node connectivity
    E = input.E[elem_num-1]
    Nu = input.Nu[elem_num-1]
    element = input.ELEM[elem_num-1] # returns the proper row of the ELEM matrix, which contains the associated nodes
    n1, n2, _, n4 = element # saves the bottom left, bottom right, and top left node numbers

    # calculate element dimensions
    a = input.NODE[n2][0] - input.NODE[n1][0] # subtract the x coordinate of n1 from the x coordinate of n2 to get x length of element
    b = input.NODE[n4][1] - input.NODE[n1][1] # subtract the y coordinate of n1 from the y coordinate of n4 to get y length of element

    # initialize symbolic variables x and y
    x = sp.Symbol('x')
    y = sp.Symbol('y')

    # define shape functions for 8 DOF rectangular plane stress/strain finite element
    f1 = (1 - (x / a)) * (1 - (y / b))
    f2 = (x / a) * (1 - (y / b))
    f3 = x * y / (a * b)
    f4 = (1 - (x / a)) * (y / b)
    shape_function_list = [f1, f2, f3, f4]
    
    def get_shape_functions(i,j):
        """return the shape function's partial derivatives"""
        fi = shape_function_list[i-1]
        fj = shape_function_list[j-1]
        dfidx = sp.diff(fi,x) # dfi/dx
        dfjdx = sp.diff(fj,x) # dfj/dx
        dfidy = sp.diff(fi,y) # dfi/dy
        dfjdy = sp.diff(fj,y) # dfj/dy
        return dfidx, dfjdx, dfidy, dfjdy
    
    def double_integral(integrand):
        """symbolic double integration over the area of the 2D element"""
        I1 = sp.integrate(integrand,(x, 0, a)) #integrate with respect to x
        solved_integral = sp.integrate(I1, (y, 0, b)) #integrate with respect to y
        return solved_integral

    # initialize quadrant kuu, kuv, and kvv matrices
    kuu = np.zeros((4,4))
    kuv = np.zeros((4,4))
    kvv = np.zeros((4,4))

    # calculate constant that is pulled out of the area integrals
    multiplier = input.t * E/(1-(Nu**2))

    # compute the local stiffness matrix by iterating through each relative index location in the four quadrant matrices of the local stiffness matrix  
    for i in range(1,5):
        for j in range(1,5):
            dfidx, dfjdx, dfidy, dfjdy = get_shape_functions(i,j) # determine shape function partial derivatives
            kuu[i-1,j-1] = multiplier * double_integral((dfidx * dfjdx) + (((1 - Nu) / 2) * dfidy * dfjdy))
            kvv[i-1,j-1] = multiplier * double_integral((dfidy * dfjdy) + (((1 - Nu) / 2) * dfidx * dfjdx))
            kuv[i-1,j-1] = multiplier * double_integral((Nu * dfidx * dfjdy) + (((1 - Nu) / 2) * dfidy * dfjdx))

    # form full local stiffness matrix by concatenating quadrant matrices
    LK = np.block([[kuu, kuv], [kuv, kvv]]) 
    return LK

In [22]:
"""module for computing the global stiffness matrix"""

import input as ip
import local_stiffness as lk
import numpy as np
import pandas as pd

def global_stiffness(): # defines a function to be called to calculate the global stiffness matrix
    """
    calculate the global stiffness matrix

    returns:
        numpy.ndarray: the global stiffness matrix
    """
    # initialize global stiffness matrix
    GK = np.zeros((2 * ip.tnnd, 2 * ip.tnnd))

    # iterate through each element and store its contribution to the global stiffness matrix
    for k in range(1,ip.tnel+1):
        # precompute the local stiffness matrix
        LK = lk.local_stiffness(k)  

        # determine element node connectivity
        element = ip.ELEM[k-1]

        # iterate through all 8 DOF of the element
        for i in range(1,9):
            # determine the row index (ii) in the global stiffness matrix
            if i <= 4: # within upper half or Fx portion of the local stiffness matrix
                ii = element[i-1] # sets the GK row index ii to be the node number of the given LK row 
            else: # within the bottom half or Fy portion of the local stiffness matrix
                i -= 4 # subtracts 4 from i so now Fy1 through Fy4 can be examined
                ii = element[i-1] + ip.tnnd # adds the total number of nodes to account for the shift to the bottom half of GK
            
            # iterate through all 8 DOF of the element
            for j in range(1,9):
                # determine the column index (jj) in the global stiffness matrix
                if j <= 4: # within the left half or u portion of the local stiffness matrix
                    jj = element[j-1] # sets the GK column index jj to be the node number of the given LK column 
                else: # within the right half or v portion of the local stiffness matrix
                    j -= 4 # subtracts 4 from j so now v1 through v4 can be examined
                    jj = element[j-1] + ip.tnnd # adds the total number of elements to account for the shift to the right half of GK
                
                # add the contribution of the local stiffness matrix to the current value in the global stiffness matrix at the computed indeces
                GK[ii-1,jj-1] += LK[i-1,j-1] # adds LK(i,j) to current value in GK
    
    return GK

if __name__ == "__main__":
    """
    will only run when the script is executed directly, not when it's imported as a module in another script
    """
    # calculate the global stiffness matrix
    GK = global_stiffness()

    # convert the global stiffness matrix to a data frame for visualization
    gk_visualization = pd.DataFrame(GK)

In [23]:
print('Global stiffness matrix (GK):\n')
gk_visualization

Global stiffness matrix (GK):



,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,436507.936508,-70207.570208,0.000000,-148046.398046,-218253.968254,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
1,-70207.570208,-148046.398046,-148046.398046,-218253.968254,292277.167277,2.922772e+05,0.000000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
2,0.000000,-148046.398046,-584554.334554,0.000000,292277.167277,4.403236e+05,0.000000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
3,-148046.398046,-218253.968254,0.000000,436507.936508,-70207.570208,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
4,-218253.968254,292277.167277,292277.167277,-70207.570208,-584554.334554,-7.783883e+04,148046.398046,218253.968254,0.000000e+00,0.000000e+00,...,148046.398046,218253.968254,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
5,0.000000,292277.167277,440323.565324,0.000000,-77838.827839,-1.021062e+06,218253.968254,148046.398046,0.000000e+00,0.000000e+00,...,218253.968254,148046.398046,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,148046.398046,2.182540e+05,-873015.873016,140415.140415,1.480464e+05,2.182540e+05,...,-873015.873016,140415.140415,1.480464e+05,2.182540e+05,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,218253.968254,1.480464e+05,140415.140415,-873015.873016,2.182540e+05,1.480464e+05,...,140415.140415,-873015.873016,2.182540e+05,1.480464e+05,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,148046.398046,218253.968254,-1.021062e+06,-7.783883e+04,...,148046.398046,218253.968254,-1.021062e+06,-7.783883e+04,0.000000,440323.565324,2.922772e+05,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,218253.968254,148046.398046,-7.783883e+04,-1.021062e+06,...,218253.968254,148046.398046,-7.783883e+04,-1.021062e+06,0.000000,292277.167277,4.403236e+05,0.000000,0.000000,0.000000
